 # <center> Penerapan Language Model pada Dataset terkait Tingkat Kekerasan yang Terjadi pada Perempuan dan Anak-anak Selama Pandemi Covid-19 </center>

**Nama  : Nadhia Salsabila Azzahra**
<br>
**NIM   : 1301174653**
<br>
**Kelas : NLP-GAB-02**

## Read Data

Data masukan berupa 20 text file berbeda yang terdiri dari judul dan isi yang dipisahkan oleh tag <kalimat><\kalimat>. Pembacaan file menggunakan bantuan library BeautifulSoup yang digunakan untuk memisahkan data antara judul dan isi dengan menggunakan *keyword* **"Kalimat"**. Pada *assignment* ini yang data yang diperlukan hanya isi artikelnya saja. 

In [1]:
from bs4 import BeautifulSoup as BS
import numpy as np
import re
import nltk
import os
import pandas as pd

In [2]:
#membaca seluruh file yang ada pada folder dataset
folderpath = r"C:\Python38\Scripts\nlp\dataset2"
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [113]:
data=[]
for path in filepaths :
    file1 = open(path,"r").read()
    soup = BS(file1)
    for item in soup.find_all("kalimat"):
        data.append(item.text[1:-1])

In [114]:
#mengambil data hanya isi artikelnya saja tanpa judul
isi=''
for artikel in (data[1:][::2]) :
    isi += artikel

In [115]:
isi

''

In [97]:
# file2 = open('summarylagi.txt', 'w')
# file2.write(str(isi))
# file2.close() 

## PREPROCESSING

Berikut merupakan tahap-tahap yang dilakukan pada proses preprocessing.
* **Lower Text** : Merubah seluruh kalimat yang ada menjadi lowercase dengan tujuan agar tidak terjadi ambiguitas kata (cth : Saya dan saya).
* **Remove New Line** : Menghilangan baris baru atau '\n' agar memudahkan proses pembangunan model bahasa (cth: 'saya\n suka' -> 'saya suka'). 
* **Remove Excessive Whitespace** : Menghilangan spasi yang berlebihan (cth: 'saya suka', 'saya suka').
* **Tokenisasi** : Memecah kalimat menjadi token-token.

In [7]:
def lowertext(text):
    return text.lower()
isi = lowertext(isi)

In [8]:
def remove_newline(text):
    return re.sub("\n"," ", text)
isi = remove_newline(isi)

In [9]:
def remove_excessive_whitespace(text):
    return re.sub('  +', ' ', text)
isi = remove_excessive_whitespace(isi)

In [10]:
def tokenize_text(isi):
    sent_text = nltk.sent_tokenize(isi)
    # loop per kalimat, tambahkan tag <s> di awal kalimat dan </s> di akhir kalimat
    mod_sentences = [] # array/list untuk menampung semua kalimat teks awal
    for sentence in sent_text:
        mod_sentence = []
        sent_tokens = nltk.word_tokenize(sentence)
        mod_sentence.append('<s>')
        for token in sent_tokens:
          mod_sentence.append(token)
        mod_sentence.append('</s>')
        mod_sentences.append(mod_sentence)
    return mod_sentences

In [11]:
isi = tokenize_text(isi)

## UNIGRAM

In [12]:
freq_tab = {}
total_count = 0
for sentence in isi:
  for token in sentence:
    if token in freq_tab:
        freq_tab[token] += 1 # kata sudah ada di dictionary, update frekuensinya
    else:
        freq_tab[token] = 1 # kata belum ada di dictionary 
    total_count += 1

In [13]:
print(total_count)
print(len(freq_tab))

17042
2571


In [14]:
prob_tab = {}
for token in freq_tab:
    prob_tab[token] = freq_tab[token]/total_count

In [15]:
test_sentence = "Jika mengalami kekerasan"
lc_test_sentence = test_sentence.lower()
test_tokens = nltk.word_tokenize(lc_test_sentence)
total_prob = 1.0
for test_token in test_tokens:
    total_prob = total_prob * prob_tab[test_token]
    print(test_token)
    
print(total_prob)

jika
mengalami
kekerasan
3.876388392120931e-08


## BIGRAM

In [16]:
freq_bigram_tab = {}
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_bigram = (sentence[i-1], sentence[i])
    if curr_bigram in freq_bigram_tab:
      freq_bigram_tab[curr_bigram] += 1 # bigram sudah ada di dictionary, update frekuensinya
    else:
      freq_bigram_tab[curr_bigram] = 1 # bigram belum ada di dictionary


In [17]:
bigram_prob_tab = {}
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_bigram = (sentence[i-1], sentence[i])

    if curr_bigram not in bigram_prob_tab:  
      bigram_prob_tab[curr_bigram] = freq_bigram_tab[curr_bigram]/freq_tab[sentence[i-1]] # bigram belum ada di dictionary, hitung probability  

## LAPLACE SMOOTHING

In [18]:
from operator import itemgetter 

In [19]:
def getList(dict): 
    return list(map(itemgetter(0), dict.items())) 

get_freq_tab = getList(freq_tab)

In [20]:
tab_freq_bigram_ls = {}

In [21]:
for i in (get_freq_tab):
    for j in (get_freq_tab):
        curr_bgram = (i,j)
        if curr_bgram not in tab_freq_bigram_ls :
            tab_freq_bigram_ls[curr_bgram] = 1
        else :
            continue

In [22]:
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_bigram = (sentence[i-1], sentence[i])
    if curr_bigram in tab_freq_bigram_ls:
      tab_freq_bigram_ls[curr_bigram] = freq_bigram_tab[curr_bigram]+1

In [24]:
tab_prob_bigram_ls = {}
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_bigram = (sentence[i-1], sentence[i])

    if curr_bigram not in tab_prob_bigram_ls:
        tab_prob_bigram_ls[curr_bigram] = tab_freq_bigram_ls[curr_bigram]/ (freq_tab.get(sentence[i-1]) + len(freq_tab)) # bigram belum ada di dictionary, hitung probability 

## PERPLEXITY FOR TEST SET

In [86]:
text1 = 'kpai menggelar survei kepada anak dan orangtua secara terpisah untuk tau kekerasan kepada anak saat selama pandemi covid-19. Harusnya lebih banyak di rumah jadi saat terbaik untuk berkumpul kembali. namun orangtua tetap bekerja meski di rumah. hal inilah yang menyebabkan kekerasan pada anak'
text2 = "belum ada keputusan resmi soal rencana relaksasi pajak ini, pun berapa usulan insentif yang dikabulkan pemerintah. tapi kalau benar terjadi tentu akan ada koreksi harga jual mobil saat ini. namun sebagai catatan, untuk pajak kendaraan bermotor itu tergantung dari masing-masing daerah. artinya bisa jadi toyota rush di satu daerah bisa lebih murah dibandingkan daerah lainnya."

In [87]:
print("data uji 1 :"+ text1)
print("data uji 2 :"+ text2)

data uji 1 :kpai menggelar survei kepada anak dan orangtua secara terpisah untuk tau kekerasan kepada anak saat selama pandemi covid-19. Harusnya lebih banyak di rumah jadi saat terbaik untuk berkumpul kembali. namun orangtua tetap bekerja meski di rumah. hal inilah yang menyebabkan kekerasan pada anak
data uji 2 :belum ada keputusan resmi soal rencana relaksasi pajak ini, pun berapa usulan insentif yang dikabulkan pemerintah. tapi kalau benar terjadi tentu akan ada koreksi harga jual mobil saat ini. namun sebagai catatan, untuk pajak kendaraan bermotor itu tergantung dari masing-masing daerah. artinya bisa jadi toyota rush di satu daerah bisa lebih murah dibandingkan daerah lainnya.


In [55]:
similar_sent=tokenize_text(text1)

In [56]:
unsimilar_sent=tokenize_text(text2)

In [57]:
def count_prob_all(test_sentences) :
    prob_all = 1.0
    for sentence in test_sentences:
        for i in range (1, len(sentence)):
            test_bigram = (sentence[i-1], sentence[i])
            if test_bigram in tab_prob_bigram_ls:
                prob_all=prob_all*tab_prob_bigram_ls.get(test_bigram) # bigram sudah ada di dictionary, update frekuensinya
            else :
                if test_bigram not in freq_tab:
                    freq_tab[sentence[i-1]]=1
                prob_all= prob_all*(freq_tab[sentence[i-1]] / (freq_tab[sentence[i-1]] + len(freq_tab)))
    return prob_all

In [58]:
def perplexity(sentence,all_prob):
    return pow(1/all_prob, (1/(float(len(sentence)))))

In [59]:
prob_unsimilar = count_prob_all(unsimilar_sent)

In [60]:
prob_similar = count_prob_all(similar_sent)

In [61]:
result_unsim = perplexity(unsimilar_sent, prob_unsimilar)
result_sim=perplexity(similar_sent, prob_similar)

In [62]:
print("Teks 1 dengan topik Covid-19")
print('Probabilitas :', prob_similar)
print('Perplexity :', result_sim)
print("\nTeks 2 dengan topik Saham Mobil")
print("Probabilitas :", prob_unsimilar)
print('Perplexity :', result_unsim)

Teks 1 dengan topik Covid-19
Probabilitas : 1.81840570606327e-126
Perplexity : 2.7231859870553806e+31

Teks 2 dengan topik Saham Mobil
Probabilitas : 2.3007004679679745e-200
Perplexity : 8.119606448205397e+49


In [63]:
d = {'Text': [text1, text2], 'Topik':['Kekerasan selama Covid-19','Harga saham mobil'], 'Nilai Probabilitas': [prob_similar, prob_unsimilar], 'Nilai Perplexity': [result_sim, result_unsim]}
df = pd.DataFrame(data=d)
print("berikut merupakan tabel probabilitas dan perplexity pada data latih")
df

,Text,Topik,Nilai Probabilitas,Nilai Perplexity
0,kpai menggelar survei kepada anak dan orangtua...,Kekerasan selama Covid-19,1.818406e-126,2.723186e+31
1,belum ada keputusan resmi soal rencana relaksa...,Harga saham mobil,2.300700e-200,8.119606e+49


## TRIGRAM

In [85]:
#proses menghitung frekuensi trigram berdasarkan data latih
freq_trigram_tab = {}
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_trigram = (sentence[i-2], sentence[i-1], sentence[i])
    if curr_trigram in freq_trigram_tab:
      freq_trigram_tab[curr_trigram] += 1 # bigram sudah ada di dictionary, update frekuensinya
    else:
      freq_trigram_tab[curr_trigram] = 1 # bigram belum ada di dictionary

In [84]:
#proses menghitung probabilitas trigram berdasarkan data latih
trigram_prob_tab = {}
for sentence in isi:
  for i in range (1, len(sentence)):
    curr_trigram = (sentence[i-2],sentence[i-1], sentence[i])

    if curr_trigram not in trigram_prob_tab:  
      trigram_prob_tab[curr_trigram] = freq_trigram_tab[curr_trigram]/(freq_tab[sentence[i-2]]+freq_tab[sentence[i-1]]) # trigram belum ada di dictionary, hitung probability  

In [67]:
#menampilkan probabilitas trigram
# print(trigram_prob_tab) 

{('</s>', '<s>', 'kpai'): 0.0014727540500736377,
 ('<s>', 'kpai', 'menggelar'): 0.0014326647564469914,
 ('kpai', 'menggelar', 'survei'): 0.05,
 ('menggelar', 'survei', 'kepada'): 0.07692307692307693,
 ('survei', 'kepada', 'anak'): 0.05405405405405406,
 ('kepada', 'anak', 'dan'): 0.007978723404255319,
 ('anak', 'dan', 'orangtua'): 0.004267425320056899,
 ('dan', 'orangtua', 'secara'): 0.0025575447570332483,
 ('orangtua', 'secara', 'terpisah'): 0.012345679012345678,
 ('secara', 'terpisah', 'untuk'): 0.021739130434782608,
 ('terpisah', 'untuk', 'mengetahui'): 0.008403361344537815,
 ('untuk', 'mengetahui', 'kekerasan'): 0.008403361344537815,
 ('mengetahui', 'kekerasan', 'kepada'): 0.0033222591362126247,
 ('kekerasan', 'kepada', 'anak-anak'): 0.003105590062111801,
 ('kepada', 'anak-anak', 'saat'): 0.014492753623188406,
 ('anak-anak', 'saat', 'selama'): 0.010309278350515464,
 ('saat', 'selama', 'pandemi'): 0.007462686567164179,
 ('selama', 'pandemi', 'covid-19'): 0.09223300970873786,
 ('pande

In [83]:
#fungsi ini digunakan untuk proses prediksi, yaitu mencari probabilitas trigram tertinggi 
def get_max_trigram_prob(prev_words):
    prev_words = prev_words.split()
    max_prob_next_word = ''
    trigram_cand = {k: v for k, v in trigram_prob_tab.items() if (k[0]==prev_word[0] and k[1]==prev_word[1])}
    if len(trigram_cand) > 0:
        trigram_cand = sorted(trigram_cand.items(), key=lambda x: x[1], reverse=True)
        k = trigram_cand[0]
        max_prob_next_word = k[0][2]
    return max_prob_next_word

In [70]:
kalimat_prediksi = "dalam survei" #masukkan inputan sesuai dengan yang diinginkan, terdiri dari 2 kata
print("kalimat prediksi trigram : " + kalimat_prediksi)

kalimat prediksi trigram : dalam survei


In [71]:
#menampilkan kata yang di prediksi sesuai dengan inputan @kalimat_prediksi
predicted_next_word = get_max_trigram_prob(kalimat_prediksi)
if len(predicted_next_word) > 0:
  print('prediksi kata selanjutnya setelah kalimat '+'\"'+kalimat_prediksi+"\""+' adalah '+"\""+predicted_next_word+"\"")
else:
  print('kata sebelum tidak ditemukan di vocabulary')

prediksi kata selanjutnya setelah kalimat "dalam survei" adalah "secara"


## EVALUASI

In [73]:
top10_unigram = sorted(freq_tab.items(), key=lambda x: x[1], reverse=True)

In [79]:
print("Berikut merupakan kata yang paling sering muncul pada unigram")
print(top10_unigram[:15])

Berikut merupakan kata yang paling sering muncul pada unigram
[(',', 1001), ('<s>', 679), ('</s>', 679), ('.', 674), ('yang', 379), ('dan', 352), ('anak', 351), ('kekerasan', 297), ('di', 271), ('perempuan', 176), ('pada', 143), ('dari', 135), ('kasus', 133), ('rumah', 130), ('pandemi', 125)]


In [81]:
top10_bigram = sorted(tab_prob_bigram_ls.items(), key=lambda x: x[1], reverse=True)

In [82]:
print("Berikut merupakan kata dengan probabilitas tertinggi pada bigram")
print(top10_bigram[:15])

Berikut merupakan kata dengan probabilitas tertinggi pada bigram
[(('.', '</s>'), 0.2061633281972265), (('orang', 'tua'), 0.02478407810739767), (('<s>', '``'), 0.024307692307692308), (('di', 'rumah'), 0.02322308233638283), (('pandemi', 'covid-19'), 0.02188427299703264), ((',', "''"), 0.02127659574468085), (('kasus', 'kekerasan'), 0.021079881656804734), (('kekerasan', 'terhadap'), 0.018479776847977684), (('perempuan', 'dan'), 0.017837641062977794), ((',', 'dan'), 0.017637178051511757), (('perlindungan', 'anak'), 0.014812001519179644), (('anak', '.'), 0.014373716632443531), (('masa', 'pandemi'), 0.014100609756097561), (('pada', 'anak'), 0.01400147383935151), (('selama', 'pandemi'), 0.013951734539969835)]
